In [1]:
import pandas as pd
from pyntcloud import PyntCloud
import pdal
import json

In [2]:
def get_points(bounds, file):
    # make pipelines to input 
    _pipeline = {
        "pipeline": [
            file,  # input las file
            {
                "type": "filters.crop",  # The crop filter removes points that fall outside or inside a cropping bounding box 
                "bounds": f"([{bounds[0]},{bounds[1]}],[{bounds[2]},{bounds[3]}])"  # input parameters for the filter, in the case the boundaries for our window
            },
            {
                "type": "writers.las", 
                "filename": "cropped_file.las" # Output file name
            }
        ]
    }
    pipeline = pdal.Pipeline(json.dumps(_pipeline))  # make pipeline to arrays to stage
    pipeline.execute()  # execute the pipeline
    assert pipeline.arrays  # validate if the pipeline works, it's expected that there will be some arrays in the pipeline result.
    point_cloud = pipeline.arrays[0]  # extract the point cloud data from the first column 
    pd_pcl = pd.DataFrame(point_cloud)[['Classification', 'X', 'Y', 'Z']]  # convert the point_cloud arrays to Panda DataFrame
    pd_pcl.columns = ['classification', 'x', 'y', 'z']  # the column names for the Panda DataFrame
    return PyntCloud(pd_pcl)  # create a PyntCloud object from the Pandas DataFrame and return it.

In [3]:
# Our boundaries
min_x = 187465.5
max_x = min_x + 500
min_y = 315228.5
max_y = min_y + 500

boundary = [min_x, max_x, min_y, max_y]

PyntCloud_ = get_points(boundary, '../data/69BZ2_13.LAZ')

In [4]:
#Checking if the boundaries have a size of 500
print(max(PyntCloud_.points['x']) - min(PyntCloud_.points['x']))
print(max(PyntCloud_.points['y']) - min(PyntCloud_.points['y']))

500.0
500.0


In [5]:
#Checking the outcome
PyntCloud_.points

,classification,x,y,z
0,1,187465.511,315228.708,167.330
1,2,187465.575,315228.546,164.819
2,1,187465.907,315228.519,166.904
3,1,187465.806,315228.811,166.978
4,2,187465.861,315228.671,164.818
...,...,...,...,...
4969633,1,187965.361,315728.260,167.263
4969634,1,187965.192,315728.420,161.977
4969635,1,187965.492,315728.152,163.846
4969636,1,187965.433,315728.241,146.752
